In [1]:
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine_csv_data')
wine #도수,당도,산성, class:0 = 화이트와인, class:1 = 레드와인

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [40]:
wine.info()

data = wine[['alcohol', 'sugar', 'pH']].to_numpy() #피쳐
target = wine['class'].to_numpy() #레이블

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [41]:
from sklearn.model_selection import train_test_split #train과 test 비율을 8:2로 나눔

x_train,x_test,y_train,y_test = train_test_split(data, target, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(5197, 3) (1300, 3) (5197,) (1300,)


## 로지스틱 회귀 모델로 학습

In [42]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# #standard 스케일러
# scaler = StandardScaler()

# scaler.fit_transform(x_train) #fit이랑 transform 같이함

#스케일러 안하니까 확률이 더 올라감

lr = LogisticRegression()
lr.fit(x_train, y_train)

print(lr.score(x_train, y_train))
print(lr.score(x_test, y_test))
print('\n')
print(lr.coef_, lr.intercept_)

0.7794881662497595
0.7761538461538462


[[ 0.42688448  0.35097153 -4.06374436]] [8.48811352]


## 결정트리 모델로 학습

In [43]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=40)
dt.fit(x_train, y_train) #여기서 x_train은 이미 위에서 스케일링함

print(dt.score(x_train, y_train)) #훈련 데이터
print(dt.score(x_test, y_test)) #테스트 데이터

0.996921300750433
0.8576923076923076


## GridSearchCV로 결정트리 하이퍼파라미터 튜닝

In [44]:
from sklearn.model_selection import GridSearchCV

estimator = DecisionTreeClassifier(random_state=42)
param_grid = {'criterion':['gini','entropy'], 'max_depth':[None,2,3,4,5,6,10], 'max_leaf_nodes':[None,2,3,4,5,6,7],
              'min_samples_split':[2,3,4,5,6], 'min_samples_leaf':[1,2,3],'splitter' : ['best', 'random'],
              'min_weight_fraction_leaf' : [0.0, 0.1, 0.2, 0.3],'min_impurity_decrease': [0.0, 0.05, 0.1, 0.2]}

gridDT = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=10, n_jobs=-1)
gridDT.fit(x_train, y_train)

print(gridDT.best_score_)
print(gridDT.best_params_)


print('\n')
print(gridDT.score(x_train, y_train))
print(gridDT.score(x_test, y_test))

0.8664580554320439
{'criterion': 'entropy', 'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


0.996921300750433
0.8507692307692307


In [58]:
from sklearn.metrics import accuracy_score

predY = gridDT.predict(x_test)
print('정확도: {:.2f}%'.format(accuracy_score(y_test, predY)*100))

정확도: 85.08%
